In [ ]:
import logging
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters
import ollama
import nest_asyncio

nest_asyncio.apply()

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

TOKEN = 'ВОТ ТЫ И ПОПАЛСЯ'

user_ids = {}
context_memory = {}

async def start(update: Update, context) -> None:
    await update.message.reply_text('Привет! Я чат-бот. Чем могу помочь?')

async def handle_message(update: Update, context):
    user_id = update.effective_user.id
    if user_id not in user_ids:
        user_ids[user_id] = {'last_message': None, 'preferences': {}}
        context_memory[user_id] = []

    message_text = update.message.text
    context_messages = context_memory[user_id]

    context_messages.append({'role': 'user', 'content': message_text})

    context_memory[user_id] = context_messages[-8:]

    try:
        response = ollama.chat(model='deepseek-r1:14b', messages=context_memory[user_id])
        
        await update.message.reply_text(response['message']['content'])
    except Exception as e:
        logging.error(f"Error while getting response from ollama: {e}")
        await update.message.reply_text('Ошибка')

async def main() -> None:
    application = ApplicationBuilder().token(TOKEN).build()

    application.add_handler(CommandHandler('start', start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    await application.run_polling()

if __name__ == '__main__':
    import asyncio
    asyncio.run(main())

2025-03-30 16:35:25,714 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7971151921:AAGpMHInb8sT8dEPSwzU37m8Fk6-WT_3b8k/getMe "HTTP/1.1 200 OK"
2025-03-30 16:35:25,971 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7971151921:AAGpMHInb8sT8dEPSwzU37m8Fk6-WT_3b8k/deleteWebhook "HTTP/1.1 200 OK"
2025-03-30 16:35:25,972 - telegram.ext.Application - INFO - Application started
2025-03-30 16:35:36,770 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7971151921:AAGpMHInb8sT8dEPSwzU37m8Fk6-WT_3b8k/getUpdates "HTTP/1.1 200 OK"
2025-03-30 16:35:38,024 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7971151921:AAGpMHInb8sT8dEPSwzU37m8Fk6-WT_3b8k/getUpdates "HTTP/1.1 200 OK"
2025-03-30 16:35:40,052 - root - ERROR - Error while getting response from ollama: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download
2025-03-30 16:35:40,840 - httpx - INFO - HTTP Request: POST ht